In [1]:
import torch
import torch.tensor
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from torch.autograd import Variable
import torch.utils.data as Data
torch.manual_seed(1)

In [2]:
VOCAB_SIZE = 4879
EMBEDDING_SIZE = 200
HIDDEN_SIZE = 200
TARGET_SIZE = 2
SENTENCE_LEN = 155
BATCH_SIZE = 128
POSITION_SIZE = 301

In [3]:
def get_myembed():
    myembed = []
    with open("word_vector.txt","r") as f:
        for line in f:
            words = line.split()
            for word in words:
                myembed.append(float(word))
    for i in range(EMBEDDING_SIZE):myembed.append(0)
    myembed = np.array(myembed)
    myembed = myembed.reshape((VOCAB_SIZE,EMBEDDING_SIZE))
    myembed = torch.from_numpy(myembed)
    return myembed

In [4]:
print(get_myembed().shape)

torch.Size([4879, 200])


In [5]:
def get_posembed(typee):
    posembedA = []
    with open(typee+"_pos_A.txt","r") as f:
        for line in f:
            words = line.split()
            temp = []
            for word in words:
                if(int(word) == 1000):word = 300
                if(int(word) == 152):word = 300
                temp.append(int(word))
            posembedA.append(temp)
    posembedB = []
    with open(typee +"_pos_B.txt","r") as f:
        for line in f:
            words = line.split()
            temp = []
            for word in words:
                if(int(word) == 1000):word = 300
                if(int(word) == 152):word = 300
                temp.append(int(word))
            posembedB.append(temp)
    return posembedA,posembedB
            

In [6]:
ansa,ansb = get_posembed("train")


In [7]:


def position_encoding_init(n_position, emb_dim):
    ''' Init the sinusoid position encoding table '''
    
    # keep dim 0 for padding token position encoding zero vector
    position_enc = np.array([
        [pos / np.power(10000, 2 * (j // 2) / emb_dim) for j in range(emb_dim)]
        if pos != 0 else np.zeros(emb_dim) for pos in range(n_position)])
    

    position_enc[1:, 0::2] = np.sin(position_enc[1:, 0::2]) # dim 2i
    position_enc[1:, 1::2] = np.cos(position_enc[1:, 1::2]) # dim 2i+1
    
   # position_enc = np.random.rand(n_position,emb_dim)
    return torch.from_numpy(position_enc).float()


In [8]:
def get_sinusoid_encoding_table(n_position, d_hid):
    ''' Sinusoid position encoding table '''

    def cal_angle(position, hid_idx):
        return position / np.power(10000, 2 * (hid_idx // 2) / d_hid)

    def get_posi_angle_vec(position):
        return [cal_angle(position, hid_j) for hid_j in range(d_hid)]

    sinusoid_table = np.array([get_posi_angle_vec(pos_i - 152) for pos_i in range(n_position)])

    sinusoid_table[:, 0::2] = np.sin(sinusoid_table[:, 0::2])  # dim 2i
    sinusoid_table[:, 1::2] = np.cos(sinusoid_table[:, 1::2])  # dim 2i+1


    
    return torch.FloatTensor(sinusoid_table)

In [9]:
hh = position_encoding_init(301,200)
print(hh[300])

tensor([-0.9998, -0.0221, -0.2809, -0.9597, -0.9742, -0.2255,  0.9816,
         0.1910,  0.2028,  0.9792,  0.7115,  0.7027,  0.1550, -0.9879,
         0.3547,  0.9350, -0.7982,  0.6024, -0.8372,  0.5469,  0.0516,
         0.9987,  0.8585, -0.5128, -0.9290,  0.3701,  0.4861, -0.8739,
         0.8108,  0.5854, -0.0416,  0.9991, -0.3793,  0.9253, -0.1524,
         0.9883,  0.5771,  0.8167,  0.9560, -0.2934, -0.4104, -0.9119,
        -0.5803,  0.8144,  0.9617, -0.2742, -0.9982, -0.0603,  0.9957,
         0.0922, -0.9880,  0.1543,  0.7920, -0.6106, -0.1789,  0.9839,
        -0.6934, -0.7206,  0.9445, -0.3284,  0.0791,  0.9969, -0.9999,
        -0.0156, -0.0363, -0.9993,  0.9755, -0.2199,  0.5048,  0.8633,
        -0.5836,  0.8120, -0.9947, -0.1030, -0.4874, -0.8732,  0.3569,
        -0.9342,  0.9176, -0.3974,  0.9498,  0.3130,  0.5559,  0.8313,
        -0.0153,  0.9999, -0.5369,  0.8436, -0.8771,  0.4803, -0.9991,
         0.0423, -0.9301, -0.3673, -0.7265, -0.6872, -0.4486, -0.8937,
      

In [10]:
class SimpleLSTM(nn.Module):
    def __init__(self,vocab_size,embedding_size,pos_size,hidden_size,target_size):
        super(SimpleLSTM,self).__init__()
        self.embed = nn.Embedding(vocab_size,embedding_size,padding_idx = 4878)
        #self.embed.requires_grad=False
        self.pos1 = nn.Embedding(pos_size,embedding_size,padding_idx = 300)
        self.pos2 = nn.Embedding(pos_size,embedding_size,padding_idx = 300)
        self.pos1.requires_grad= True
        self.pos2.requires_grad= True
        self.lstm = nn.LSTM(embedding_size ,hidden_size,batch_first = True)
        self.linear = nn.Linear(hidden_size,target_size)
        self.dropout = nn.Dropout(p=0.5)
        self.myinit(embedding_size,pos_size)
    def myinit(self,embedding_size,pos_size):
        
        self.pos1.weight.data.copy_(get_sinusoid_encoding_table(pos_size, embedding_size))
        self.pos2.weight.data.copy_(get_sinusoid_encoding_table(pos_size, embedding_size))
        self.embed.weight.data.copy_(get_myembed())
        #self.pos1.weight.data.copy_(position_encoding_init(pos_size, embedding_size))
        #self.pos2.weight.data.copy_(position_encoding_init(pos_size, embedding_size))
       # nn.init.xavier_uniform_(self.pos1.weight.data)
        if self.pos1.padding_idx is not None:
            self.pos1.weight.data[self.pos1.padding_idx].fill_(0)
           # self.pos1.weight.data[152].fill_(0)
        #nn.init.xavier_uniform_(self.pos2.weight.data)
        if self.pos2.padding_idx is not None:
            self.pos2.weight.data[self.pos2.padding_idx].fill_(0)
            #self.pos2.weight.data[152].fill_(0)
        
       # self.init_pos_weights()

    def forward(self,sentence,inpos1,inpos2):
        
        myembed = self.embed(sentence)
       # print(myembed.shape)
        mypos1 = self.pos1(inpos1)
        #print(mypos1.shape)
        mypos2 = self.pos2(inpos2)
        #nextt = torch.cat((myembed,mypos1,mypos2),2)
        #print(nextt.shape)
        nextt = myembed + mypos1 + mypos2
        output,(hidden,c) = self.lstm(nextt.view(-1,SENTENCE_LEN,EMBEDDING_SIZE ))
        
        predict = self.linear((output[:,-1,:].view(-1,HIDDEN_SIZE)))
        return predict        

In [11]:
def load_data(typee):
    origin_y = []
    with open(typee + "_input_label.txt","r") as f:
        for line in f:
            origin_y.append(int(line))
    origin_x = []
    with open(typee + "_input_pad.txt","r") as f:
        for line in f:
            words = line.split()
            if len(words)!= SENTENCE_LEN:
                print(len(words))
           # assert len(words) == SENTENCE_LEN,"sentence pad"
            for word in words:
                origin_x.append(int(word))
    numpx = np.array(origin_x)
    numpx = numpx.reshape(-1,SENTENCE_LEN)
    x = torch.from_numpy(numpx)   
    numpy = np.array(origin_y)
    y = torch.from_numpy(numpy)
    x_pos_A,x_pos_B = get_posembed(typee)
    x_pos_A = np.array(x_pos_A)
    x_pos_A = torch.from_numpy(x_pos_A)
    x_pos_B = np.array(x_pos_B)
    x_pos_B = torch.from_numpy(x_pos_B)
    new_x = torch.cat((x,x_pos_A,x_pos_B),1)

    return new_x,y

In [12]:
tp = 0
fp = 0
fn = 0

In [13]:
def evaluate(predict,y):
    global tp 
    global fp 
    global fn
    tp += torch.sum(torch.masked_select(mask = y.byte(),input = predict))
    fp += torch.sum(torch.masked_select(mask =~y.byte(),input = predict))
    fn += torch.sum(torch.masked_select(mask =y.byte(),input = ~predict.byte()))

In [14]:
'''
predict = np.array([0,1,0,0,1,1,1])
y = np.array      ([1,0,1,1,0,0,0])
predict = torch.from_numpy(predict)
y = torch.from_numpy(y)
evaluate(predict,y)
'''

'\npredict = np.array([0,1,0,0,1,1,1])\ny = np.array      ([1,0,1,1,0,0,0])\npredict = torch.from_numpy(predict)\ny = torch.from_numpy(y)\nevaluate(predict,y)\n'

In [15]:
def whole_eval():
    precision = tp.float() / (tp.float() + fp.float())
    recall = tp.float() / (tp.float() + fn.float())
    print(precision)
    print(recall)
    f1 = 2 * precision * recall / (precision + recall)
    return f1
            

In [16]:

x,y = load_data("train")


In [17]:

from collections import Counter

import numpy as np


from sklearn.datasets import load_iris

from imblearn.datasets import make_imbalance

from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.under_sampling import TomekLinks

In [18]:
'''
sampling_strategy = {0: 4021, 1: 4021}
bx,by = make_imbalance(x, y,sampling_strategy=sampling_strategy)
print(bx,by)
bx = torch.from_numpy(bx)
by = torch.from_numpy(by)
'''

'\nsampling_strategy = {0: 4021, 1: 4021}\nbx,by = make_imbalance(x, y,sampling_strategy=sampling_strategy)\nprint(bx,by)\nbx = torch.from_numpy(bx)\nby = torch.from_numpy(by)\n'

In [19]:

sampling_strategy = 'not majority'

ros = RandomOverSampler(sampling_strategy=sampling_strategy)
X_res, y_res = ros.fit_resample(x, y)
print(X_res.shape)
print(y_res.shape)
X_res = torch.from_numpy(X_res)
y_res = torch.from_numpy(y_res)

(47542, 465)
(47542,)


In [20]:
#help((torch.cat))



In [60]:
torch_dataset = Data.TensorDataset(x,y)
#torch_dataset = Data.TensorDataset(X_res,y_res)
loader = Data.DataLoader(
    dataset = torch_dataset,
    batch_size = BATCH_SIZE,
    shuffle = True,
    num_workers = 2
)

In [30]:

tx,ty = load_data("test")
#torch_dataset_test = Data.TensorDataset(tx,ty)
torch_dataset = Data.TensorDataset(X_res,y_res)
loader_test = Data.DataLoader(
    dataset = torch_dataset_test,
    batch_size = BATCH_SIZE,
    shuffle = True,
    num_workers = 2
)


In [23]:
print(np.array(ty).shape)

(5761,)


In [46]:

mynet = SimpleLSTM(VOCAB_SIZE,EMBEDDING_SIZE,POSITION_SIZE,HIDDEN_SIZE,TARGET_SIZE)
mynet.cuda()
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(mynet.parameters(), lr=0.01)

his_loss = []


In [59]:
optimizer = optim.Adam(mynet.parameters(), lr=0.001)

In [61]:

wrongnum = 0
totalone = 0
for epoch in range(10):
    global tp
    global fp
    global fn
    tp = 0
    fp = 0
    fn = 0
    wrongnum = 0
    totalone = 0
    whole_loss = 0.0
    print(len(loader))
    for step,(batch_x,batch_y)in enumerate(loader):
        #if(step % 100) == 0:print(step)
        #print(batch_x.shape)
        batch_x = batch_x.cuda()
        batch_y = batch_y.cuda()
        sentence = batch_x[:,:SENTENCE_LEN]
        pos1 = batch_x[:,SENTENCE_LEN:SENTENCE_LEN + SENTENCE_LEN]
        pos2 = batch_x[:,SENTENCE_LEN + SENTENCE_LEN:]
        out = mynet(sentence,pos1,pos2)
        pred = torch.max(F.softmax(out,dim = 1),1)[1]
        wrongnum += torch.sum(pred != batch_y)
        totalone += torch.sum(pred)
        loss = loss_function(out,batch_y)
        evaluate(pred.cuda(),batch_y)
        whole_loss += loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print("--------------------------------------")
    print(wrongnum)
    print(totalone)
    print(whole_loss)
    his_loss.append(whole_loss)
    print("---------------")
    print(tp)
    print(fp)
    print(fn)
    print("!!!!!!!!!!!!!!!!!!!")
    test()

218
--------------------------------------
tensor(3954, device='cuda:0')
tensor(345, device='cuda:0')
tensor(81.8291, device='cuda:0')
---------------
tensor(206, device='cuda:0')
tensor(139, device='cuda:0')
tensor(3815, device='cuda:0')
!!!!!!!!!!!!!!!!!!!
0
myeval
tensor(0.5219, device='cuda:0')
tensor(0.1338, device='cuda:0')
tensor(0.2130, device='cuda:0')
218
--------------------------------------
tensor(3786, device='cuda:0')
tensor(843, device='cuda:0')
tensor(71.4896, device='cuda:0')
---------------
tensor(539, device='cuda:0')
tensor(304, device='cuda:0')
tensor(3482, device='cuda:0')
!!!!!!!!!!!!!!!!!!!
0
myeval
tensor(0.7778, device='cuda:0')
tensor(1.00000e-02 *
       1.4300, device='cuda:0')
tensor(1.00000e-02 *
       2.8084, device='cuda:0')
218
--------------------------------------
tensor(3619, device='cuda:0')
tensor(1770, device='cuda:0')
tensor(63.2891, device='cuda:0')
---------------
tensor(1086, device='cuda:0')
tensor(684, device='cuda:0')
tensor(2935, device

In [27]:
#torch.save(mynet, "my79.pkl")
#torch.save(mynet.state_dict(), "83.pa")
#newmynet = torch.load("82.pa")
#print(newmynet)
#newmynet.eval()
test()

0
myeval
tensor(0.3816, device='cuda:0')
tensor(0.6517, device='cuda:0')
tensor(0.4813, device='cuda:0')


In [26]:
def test():
    global tp
    global fp
    global fn
    tp = 0
    fp = 0
    fn = 0
    for step,(batch_x,batch_y)in enumerate(loader_test):
            batch_x = batch_x.cuda()
            batch_y = batch_y.cuda()
            if(step % 100) == 0:print(step)

            sentence = batch_x[:,:SENTENCE_LEN]
            pos1 = batch_x[:,SENTENCE_LEN:SENTENCE_LEN + SENTENCE_LEN]
            pos2 = batch_x[:,SENTENCE_LEN + SENTENCE_LEN:]
            out = mynet(sentence,pos1,pos2)
            pred = torch.max(F.softmax(out,dim = 1),1)[1]
            #print(pred)
            #print(batch_y)
            evaluate(pred.cuda(),batch_y.cuda())
    print("myeval")
    print(whole_eval())

In [ ]:
print(his_loss)
print(wrongnum)
print(totalone)

In [ ]:
print(tp)
print(fp)
print(fn)

In [ ]:
whole_eval()